## Creating a pipeline
### Want one function (eventually) to clean up text, create a dictionary, and return a DataFrame efficiently.

In [164]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style='darkgrid')
import spacy
from gensim import corpora

In [241]:
df = pd.read_csv('full.csv')
df.head()

,Unnamed: 0,ListingId,PublicRemarks,MLSListingID,ListingBrokerName,ailPropertyID,ClosedDate,ClosedPrice,avmValue,avmCompMatchLevel,...,topic_0.0,topic_1.0,topic_2.0,topic_3.0,topic_4.0,topic_5.0,Actual,Predicted,Residual,perc_improved
0,8,200488922,"Charming Brick Home. 4 bedrooms, 2 bath. This ...",1588623,Patricia K Green,77369181,2019-04-30,215000.0,216270.82,6,...,0,0,0,0,0,1,0.005876,0.240286,0.234410,-3889.238059
1,23,200609063,SINGLE LEVEL HOME updated throughout. VERY cle...,1594269,Jeffrey Boyson,77384141,2019-05-20,242000.0,241906.44,5,...,0,0,0,1,0,0,0.000387,0.064723,0.064336,-16534.500592
2,34,200691727,This beautiful newly renovated home has an ope...,1595726,Erica Peterson,76847035,2019-06-05,290000.0,252066.80,6,...,0,0,0,1,0,0,0.150489,0.072125,0.078364,47.926967
3,39,200721083,This North Ogden beauty has features even bett...,1596255,Ashley Wolthuis,77342957,2019-05-31,255000.0,258307.76,10,...,0,1,0,0,0,0,0.012805,0.093945,0.081139,-533.626801
4,53,201121130,**MULTIPLE OFFERS RECEIVED. NO MORE SHOWINGS ...,1598584,Nathan Poulsen,77293443,2019-07-11,220000.0,207384.94,10,...,0,0,0,1,0,0,0.060829,0.129453,0.068624,-12.813917


Here's what the topics are: 


0.5190946781513662

In [242]:
%%time
import logging
import gensim
import spacy
import re
from nltk.corpus import stopwords
nlp = spacy.load('en_core_web_sm')
df['PublicRemarks']=df['PublicRemarks'].astype(str)
df['tokens'] = df['PublicRemarks'].map(lambda x: nlp.tokenizer(x.lower()))

stops = [spacy.lang.en.stop_words.STOP_WORDS] + stopwords.words('english') + ['bath','baths','bed','beds','bedroom', 'bedrooms', 
                                                                     'property', 'home', 'lot', 'room', 'kitchen', 'mstr',
                                                                     'master', 'bath', 'floor', 'main', 'level', 
                                                                     'living', 'room', 'home', 'area', 'w'] + ['able', 'about', 'above', 'abst', 'accordance', 'according', 'accordingly', 'across', 'act', 'actually', 'added', 'adj', 'affected', 'affecting',
                    'affects', 'after', 'afterwards', 'again', 'against', 'ah', 'all', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among',
                   'amongst', 'an', 'and', 'announce', 'another', 'any', 'anybody', 'anyhow', 'anymore', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apparently',
                   'approximately', 'are', 'aren', 'arent', 'arise', 'around', 'as', 'aside', 'ask', 'asking', 'at', 'auth', 'away', 'awfully', 'back',
                   'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'begin', 'beginning', 'beginnings', 'begins', 'behind',
                   'being', 'believe', 'below', 'beside', 'besides', 'between', 'beyond', 'biol', 'both', 'brief', 'briefly', 'but', 'by', 'c', 'ca', 'came', 'can',
                   'cannot', 'can''t', 'cause', 'causes', 'certain', 'certainly', 'co', 'com', 'come', 'comes', 'contain', 'containing', 'contains', 'could', 'couldnt',
                   'date', 'did', 'didn''t', 'different', 'do', 'does', 'doesn''t', 'doing', 'done', 'don''t', 'down', 'downwards', 'due', 'during', 'each', 'ed', 'edu',
                   'effect', 'eg', 'eight', 'eighty', 'either', 'else', 'elsewhere', 'end', 'ending', 'enough', 'especially', 'et', 'et-al', 'etc', 'even', 'ever', 'every',
                   'everybody', 'everyone', 'everything', 'everywhere', 'except', 'far', 'few', 'ff', 'fifth', 'first', 'five', 'followed', 'following', 'follows',
                   'for', 'former', 'formerly', 'forth', 'found', 'four', 'from', 'further', 'furthermore', 'gave', 'get', 'gets', 'getting', 'give', 'given', 'gives',
                   'giving', 'go', 'goes', 'gone', 'got', 'gotten', 'had', 'happens', 'hardly', 'has', 'hasn''t', 'have', 'haven''t', 'having', 'he', 'hed', 'hence', 'her',
                   'here', 'hereafter', 'hereby', 'herein', 'heres', 'hereupon', 'hers', 'herself', 'hes', 'hi', 'hid', 'him', 'himself', 'his', 'hither', 'home', 'how',
                   'howbeit', 'however', 'hundred', 'id', 'ie', 'if', 'im', 'importance', 'important', 'in', 'inc', 'indeed', 'index',
                   'information', 'instead', 'into', 'invention', 'inward', 'is', 'isn''t', 'it', 'itd', 'it''ll', 'its', 'itself', 'just', 'keep', 'keeps', 'kept',
                   'kg', 'km', 'know', 'known', 'knows', 'largely', 'last', 'lately', 'later', 'latter', 'latterly', 'least', 'less', 'lest', 'let', 'lets', 'like', 'liked',
                   'likely', 'line', 'little', 'look', 'looking', 'looks', 'ltd', 'made', 'mainly', 'make', 'makes', 'many', 'may', 'maybe', 'me', 'mean', 'means', 'meantime',
                   'meanwhile', 'merely', 'mg', 'might', 'million', 'miss', 'ml', 'more', 'moreover', 'most', 'mostly', 'much', 'mug', 'my', 'myself', 'name', 'namely', 'nay',
                   'near', 'nearly', 'necessarily', 'neither', 'never', 'nevertheless', 'new', 'next', 'nine', 'ninety', 'no', 'nobody', 'non',
                   'none', 'nonetheless', 'noone', 'nor', 'normally', 'nos', 'not', 'noted', 'nothing', 'now', 'nowhere', 'obtain', 'obtained', 'obviously', 'of', 'off', 'often',
                   'oh', 'ok', 'okay', 'old', 'omitted', 'on', 'once', 'one', 'ones', 'only', 'onto', 'or', 'ord', 'other', 'others', 'otherwise', 'ought', 'our', 'ours', 'ourselves',
                   'out', 'outside', 'over', 'overall', 'owing', 'own', 'page', 'pages', 'part', 'particular', 'particularly', 'past', 'per', 'perhaps', 'placed', 'please', 'plus',
                   'poorly', 'possible', 'possibly', 'potentially', 'predominantly', 'present', 'previously', 'primarily', 'probably', 'promptly', 'proud', 'provides', 'put', 'que',
                   'quickly', 'quite', 'qv', 'ran', 'rather', 'rd', 're', 'readily', 'really', 'recent', 'recently', 'ref', 'refs', 'regarding', 'regardless', 'regards', 'related',
                   'relatively', 'research', 'respectively', 'resulted', 'resulting', 'results', 'right', 'run', 'said', 'same', 'saw', 'say', 'saying', 'says', 'sec', 'section',
                   'see', 'seeing', 'seem', 'seemed', 'seeming', 'seems', 'seen', 'self', 'selves', 'sent', 'seven', 'several', 'shall', 'she', 'shed', 'she''ll', 'shes', 'should',
                   'shouldn''t', 'show', 'showed', 'shown', 'showns', 'shows', 'significant', 'significantly', 'similar', 'similarly', 'since', 'six', 'slightly', 'so', 'some',
                   'somebody', 'somehow', 'someone', 'somethan', 'something', 'sometime', 'sometimes', 'somewhat', 'somewhere', 'soon', 'sorry', 'specifically', 'specified', 'specify',
                   'specifying', 'still', 'stop', 'strongly', 'sub', 'successfully', 'such', 'sufficiently', 'suggest', 'sup', 'sure', 'take', 'taken', 'taking',
                   'tell', 'tends', 'th', 'than', 'thank', 'thanks', 'thanx', 'that', 'that''ll', 'thats', 'that''ve', 'the', 'their', 'theirs', 'them', 'themselves', 'then',
                   'thence', 'there', 'thereafter', 'thereby', 'thered', 'therefore', 'therein', 'there''ll', 'thereof', 'therere', 'theres', 'thereto', 'thereupon', 'there''ve',
                   'these', 'they', 'theyd', 'they''ll', 'theyre', 'they''ve', 'think', 'this', 'those', 'thou', 'though', 'thoughh', 'thousand', 'throug', 'through',
                   'thru', 'thus', 'til', 'tip', 'to', 'together', 'too', 'took', 'toward', 'towards', 'tried', 'tries', 'truly', 'try', 'trying', 'ts', 'twice', 'two', 'under',
                   'unfortunately', 'unless', 'unlike', 'unlikely', 'until', 'unto', 'up', 'upon', 'ups', 'us', 'use', 'used', 'useful', 'usefully', 'usefulness', 'uses', 'using',
                   'usually', 'various', 'very', 'via', 'viz', 'vol', 'vols', 'vs', 'want', 'wants', 'was', 'wasnt', 'way', 'we', 'wed', 'welcome', 'we''ll', 'went', 'were',
                   'werent', 'we''ve', 'what', 'whatever', 'what''ll', 'whats', 'when', 'whence', 'whenever', 'where', 'whereafter', 'whereas', 'whereby', 'wherein', 'wheres',
                   'whereupon', 'wherever', 'whether', 'which', 'while', 'whim', 'whither', 'who', 'whod', 'whoever', 'whole', 'who''ll', 'whom', 'whomever', 'whos', 'whose',
                   'why', 'widely', 'willing', 'wish', 'with', 'within', 'without', 'wont', 'words', 'world', 'would', 'wouldnt', 'www','yes', 'yet', 'you', 'youd', 'you''ll',
                   'your', 'youre', 'yours', 'yourself', 'yourselves', 'you''ve', 'zero']
from gensim.parsing.preprocessing import strip_short
from symspellpy import SymSpell, Verbosity
import pkg_resources
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

def clean_jv(doc):
   typo_free = " ".join([(sym_spell.lookup(i, Verbosity.CLOSEST, max_edit_distance=2, include_unknown=True)[0].term) for i in doc]) 
   twol_free = strip_short(typo_free)
   return twol_free
def clean(lst):
    lst = [token for token in lst if not token.is_stop]
    lst = [token.lemma_ for token in lst if str(token.lemma_) not in stops]
    lst = [re.sub(r'[\W\d\s]', '', string) for string in lst]
    lst = [token for token in lst if token not in stops]
    while '' in lst:
        lst.remove('')
    lst = clean_jv(lst)
    return lst
df['tokens'] = df['tokens'].map(lambda x: clean(x))
#Creating a single-string version of the cleaned texts, for tools that require it.
df['token_str'] = df['tokens'].map(lambda x: ' '.join(x))
doc_clean = df['tokens'].map(lambda x: list(x.split()))
bigram =gensim.models.Phrases(doc_clean, min_count = 5, threshold = 50)
trigram =gensim.models.Phrases(bigram[doc_clean], min_count = 5, threshold = 50)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
def make_bigrams(texts):
    return bigram_mod[texts]
def make_trigrams(texts):
    return trigram_mod[bigram_mod[texts]]
doc_trigram = df['tokens'].map(lambda x: make_trigrams(x.split()))
dct = corpora.Dictionary(doc_trigram)
dct.filter_extremes(no_below=5, no_above=0.6666, keep_n=90000)
doc_term_matrix = [dct.doc2bow(doc) for doc in doc_trigram]

Wall time: 1min 52s


In [262]:
from gensim.models.tfidfmodel import TfidfModel
#Create tf-idf vectors
tfidf = TfidfModel(corpus)
vectors = tfidf[corpus]
from gensim.models.nmf import Nmf
nmf_model = Nmf(vectors, id2word = dct, num_topics=6, kappa=0.08, eval_every=500, normalize=True, passes=10, random_state=42)
print("Here's what the topics are: ")
nmf_model.print_topics()
from gensim.models.coherencemodel import CoherenceModel
cm = CoherenceModel(model=nmf_model, corpus=vectors, texts = doc_trigram, coherence='c_v')
coherence = cm.get_coherence()  # get coherence value
coherence # show coherence score.

Here's what the topics are: 


0.3267375348097861

In [263]:
nmf_model.print_topics()


[(0,
  '0.009*"interior" + 0.009*"fast" + 0.008*"rich" + 0.008*"remodel" + 0.007*"glass" + 0.007*"vault" + 0.007*"spectacular" + 0.007*"afternoon" + 0.006*"backslash" + 0.006*"response"'),
 (1,
  '0.012*"concrete" + 0.011*"great" + 0.010*"acclaim" + 0.008*"hot_water_heater" + 0.008*"fully" + 0.007*"quiet" + 0.007*"wood" + 0.007*"custom" + 0.006*"walkthrough" + 0.006*"convenient"'),
 (2,
  '0.015*"suite" + 0.015*"reserve" + 0.014*"easy_commute" + 0.014*"fresh" + 0.014*"speed" + 0.014*"beach" + 0.014*"search" + 0.014*"consist" + 0.013*"electrical_plumb" + 0.013*"easy"'),
 (3,
  '0.011*"live" + 0.011*"afternoon" + 0.009*"pane" + 0.009*"condition" + 0.009*"port" + 0.008*"deck" + 0.008*"large" + 0.008*"proof" + 0.008*"clean" + 0.007*"drawer"'),
 (4,
  '0.013*"appliance" + 0.013*"update" + 0.011*"tile" + 0.010*"stave" + 0.009*"drawer" + 0.009*"central_air" + 0.009*"flow" + 0.008*"average" + 0.008*"carpet" + 0.008*"deadline"'),
 (5,
  '0.007*"backyard" + 0.007*"long" + 0.006*"wood" + 0.006*"a

In [246]:
doc_trigram[:20]

0     [charm, brick, update, feature, tile, tile, ca...
1     [single, update, clean, white, appliance, incl...
2     [beautiful, newly, renovate, open, space, conc...
3     [north, ogden, beauty, feature, stun, mountain...
4     [multiple, offer, receive, showing, time, read...
5     [split, entry, quiet, jul_sac, wait, family, l...
6     [remodel, perfect, condition, laminate, wood, ...
7     [spectacular, remodel, great, location, large,...
8     [price_reduce, beautifully, remodel, centervil...
9     [layton, enjoy, open, space, ready, entertain,...
10    [glen, eagle, golf_course, nearby, superb, upd...
11    [upper, east, bench, price_reduce, maintain, b...
12    [beautiful, mccall, build, soar, ceiling, beau...
13    [park, backyard, love, great, sip_morning_coff...
14    [dream, relax, porch, brand, fully, fence, acr...
15    [great, east, bench, great, neighbourhood, squ...
16    [large, vault, ceiling, upstairs, include, pri...
17    [darling, east, layton, wonderful, plan, i

In [260]:
trigram_mod.score_item('new', 'renovated', components=components)

NameError: name 'components' is not defined

In [261]:
doc_trigram

0        [charm, brick, update, feature, tile, tile, ca...
1        [single, update, clean, white, appliance, incl...
2        [beautiful, newly, renovate, open, space, conc...
3        [north, ogden, beauty, feature, stun, mountain...
4        [multiple, offer, receive, showing, time, read...
                               ...                        
24877    [brick, ranch, grow, family, need, grow, step,...
24878    [newly, update, beautiful, half, house, oak, h...
24879    [cute_button, describe, farmhouse, entryway, f...
24880    [warmth, abound, price, estate, sale, hardwood...
24881    [rarely_available, ranch, meticulously_maintai...
Name: tokens, Length: 24882, dtype: object